Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [6]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

## Convolutional NN

Let's build a small network with **two convolutional layers, followed by one fully connected layer**. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        '''
        The filter is applied to image patches of the same size as the filter and strided according to the strides argument. 
        strides = [1, 1, 1, 1] applies the filter to a patch at every offset, 
        strides = [1, 2, 2, 1] applies the filter to every other image patch in each dimension, etc.
        https://www.tensorflow.org/api_docs/python/nn/convolution#convolution
        tf.nn.conv2d(input, 
                     filter, 
                     strides, 
                     padding, 
                     use_cudnn_on_gpu=None, 
                     data_format=None, 
                     name=None)
        '''
        conv = tf.nn.conv2d(data, # [batch, in_height, in_width, in_channels]
                            layer1_weights, # [filter_height, filter_width, in_channels, out_channels]
                            [1, 2, 2, 1], # applies the filter to every other image patch in each dimension
                            padding = 'SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        
        conv = tf.nn.conv2d(hidden, 
                            layer2_weights, 
                            [1, 2, 2, 1], 
                            padding = 'SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, 
                             [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [20]:
num_steps = 1000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, 
                     tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, 
                                         loss, 
                                         train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('\n')
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    print('\n')

Initialized
Minibatch loss at step 0: 3.039936
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%


Minibatch loss at step 100: 1.207400
Minibatch accuracy: 50.0%
Validation accuracy: 67.0%


Minibatch loss at step 200: 0.837177
Minibatch accuracy: 75.0%
Validation accuracy: 77.2%


Minibatch loss at step 300: 0.545992
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%


Minibatch loss at step 400: 0.332291
Minibatch accuracy: 93.8%
Validation accuracy: 78.8%


Minibatch loss at step 500: 0.709164
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%


Minibatch loss at step 600: 0.528524
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%


Minibatch loss at step 700: 0.803232
Minibatch accuracy: 75.0%
Validation accuracy: 81.3%


Minibatch loss at step 800: 0.711203
Minibatch accuracy: 68.8%
Validation accuracy: 82.6%


Minibatch loss at step 900: 0.601846
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%


Test accuracy: 89.1%




---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

* ** Stride **: number of pixels we are shifting each time we move our filter. When stride = 1, the output is roughly the same size as the input(roughly because it depends on the padding). When stride = 2, the output is roughly half the size of the input. 
* **Padding **: don't go pass the edge - VALID padding, go pass the edge and fill with 0s - SAME padding
* ** Patch size **: the depth increases as move ahead in the CNN.

<img src = 'cnn.png'></img>

In [22]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    # tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
    layer_1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev = 0.1))
    layer_1_biases = tf.Variable(tf.zeros([depth]))
    
    layer_2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev = 0.1))
    layer_2_biases = tf.Variable(tf.constant(1.0, shape = [depth]))
    
    layer_3_weights = tf.Variable(tf.truncated_normal([image_size / 4 * image_size / 4 * depth, num_hidden], stddev = 0.1))
    layer_3_biases = tf.Variable(tf.constant(1.0, shape = [num_hidden]))
    
    layer_4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev = 0.1))
    layer_4_biases = tf.Variable(tf.constant(1.0, shape = [num_labels]))
    
    def model(data, train = False):
        conv_1 = tf.nn.conv2d(data, 
                            layer_1_weights,
                            [1,1,1,1], 
                            padding = 'SAME')
        hidden = tf.nn.relu(tf.nn.bias_add(conv_1, layer_1_biases))
        
        # Pooling
        max_pool_1 = tf.nn.max_pool(hidden,
                                 ksize = [1,2,2,1],
                                 strides = [1,2,2,1],
                                 padding = 'SAME')
        conv_2 = tf.nn.conv2d(max_pool_1,
                           layer_2_weights,
                           [1,1,1,1],
                           padding = 'SAME')
        hidden = tf.nn.relu(tf.nn.bias_add(conv_2, layer_2_biases))
        
        # Pooling
        max_pool_2 = tf.nn.max_pool(hidden,
                                 ksize = [1,2,2,1],
                                 strides = [1,2,2,1],
                                 padding = 'SAME')
        shape = max_pool_2.get_shape().as_list()
        reshape = tf.reshape(max_pool_2, 
                            [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer_3_weights) + layer_3_biases)
        
        return tf.matmul(hidden, layer_4_weights) + layer_4_biases
        
    
    # Training computations
    logits = model(tf_train_dataset, True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

In [24]:
num_steps = 1000

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset : (offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if step % 100 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('\n')
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.274907
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%


Minibatch loss at step 100: 1.616006
Minibatch accuracy: 50.0%
Validation accuracy: 55.1%


Minibatch loss at step 200: 0.917275
Minibatch accuracy: 68.8%
Validation accuracy: 77.9%


Minibatch loss at step 300: 0.536552
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%


Minibatch loss at step 400: 0.216105
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%


Minibatch loss at step 500: 0.780570
Minibatch accuracy: 81.2%
Validation accuracy: 82.5%


Minibatch loss at step 600: 0.438341
Minibatch accuracy: 87.5%
Validation accuracy: 81.9%


Minibatch loss at step 700: 0.661848
Minibatch accuracy: 81.2%
Validation accuracy: 83.4%


Minibatch loss at step 800: 0.617353
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%


Minibatch loss at step 900: 0.417148
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%


Test accuracy: 90.9%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---